In [22]:
from lxml.etree import tostring
from itertools import chain
from lxml import etree
import urllib2
from pandas import DataFrame
import pandas
import time
import re

In [2]:
# Trying to get info from one page

url = 'http://www.lenta.com'
test = urllib2.urlopen(url)
page = test.read()
tree = etree.HTML(page)

In [201]:
def get_promo_territories(url):
    """

    """
    page_open = urllib2.urlopen(url)
    page_read = page_open.read()
    tree = etree.HTML(page_read.decode('utf-8'))

    li = []
    table = tree.xpath(u'//script')
    i = 0
    cities = etree.tounicode(table[6])
    cities_iter = re.finditer("myAllTownInfo.*?'];",cities)
    for x in cities_iter:
        temp_terr = x.group(0)
        temp_start = temp_terr.index(']=[')
        city = temp_terr[(temp_start+3):-3].replace("'", '').split(',')
        # print city
        li.append(city)
    
    df = DataFrame(li)
    df.columns = ['Abbr','Href','City']
    #print df
    return df


In [202]:
def get_all_pages(url):
    """
    
    """
    
    li = []

    def get_one_sub(url1):
        #url = url + '/neprodovol-stvennye-tovary/'
        page_open = urllib2.urlopen(url1)
        page_read = page_open.read()
        page_read = page_read
        tree = etree.HTML(page_read)
        table = tree.xpath('//ul[@class = "sub-menu"]')
        for x in table:
            temp =  etree.tounicode(x)
            temp = temp.replace('<span>', '')
            temp = temp.replace('</span>', '')
            #print temp
            temp_tree = etree.HTML(temp)
    #        newprice = ''.join(temp_tree.xpath('//div//span[@class = "new-price"]/text()')).replace('.',',')
    #        oldprice = ''.join(temp_tree.xpath('//div//div[@class = "old-price"]/div/text()')).replace('.',',')
    #        title = ''.join(temp_tree.xpath('//div//div[@class = "title"]/text()'))
    #        bonus = ''.join(temp_tree.xpath('//div//div[@class = "bonus"]/text()'))
            sub = temp_tree.xpath('//ul/li/a[@class = "sub-menu__link"]/text()')
            href = temp_tree.xpath('//ul/li/a[@class = "sub-menu__link"]/@href')
            for xx in zip(sub, href):
                #print xx
                li.append(xx)
        
        df = DataFrame(li)
        #print df
        return df
    df_all = get_one_sub(url + '/neprodovol-stvennye-tovary/')
    #df_all = df_all.append(get_one_sub(url + '/prodovol-stvennye-tovary/'))
    df_all = get_one_sub(url + '/prodovol-stvennye-tovary/')
    #df_all.columns = ['Name', 'Href']
    return df_all

#print get_all_pages('http://www.lenta.com')

In [130]:
terrs = get_promo_territories('http://www.lenta.com')
print terrs

         Abbr                            Href                    City
0         spb            http://www.lenta.com         Санкт-Петербург
1         arm        http://www.arm.lenta.com                 Армавир
2         ast        http://www.ast.lenta.com               Астрахань
3         alm        http://www.alm.lenta.com             Альметьевск
4         blg        http://www.blg.lenta.com                Белгород
5         brn        http://www.brn.lenta.com                 Барнаул
6         bsk        http://www.bsk.lenta.com                   Бийск
7       brnsk      http://www.brnsk.lenta.com                  Брянск
8         blk        http://www.blk.lenta.com                Балаково
9        vnov       http://www.vnov.lenta.com        Великий Новгород
10        vlg        http://www.vlg.lenta.com               Волгоград
11        vlz        http://www.vlz.lenta.com                Волжский
12        vrn        http://www.vrn.lenta.com                 Воронеж
13        vol       

In [203]:
def get_one_page(url):
    """

    """
    li = []
    page_open = urllib2.urlopen(url)
    page_read = page_open.read()
    tree = etree.HTML(page_read)
    table = tree.xpath('//div[@class = "catalog_container cleara"]')
    #print etree.tounicode(table[0])
    temp_tree = etree.HTML(etree.tounicode(table[0]))
    temp_table = temp_tree.xpath('//div[@class = "catalog__item"]')
    for x in temp_table:
        sub_x = etree.tounicode(x).replace("<br/>", "").replace("<p>", "").replace("</p>", "")
        sub_x_tree = etree.HTML(sub_x)
        sub_x_name = "".join(sub_x_tree.xpath('//a[@class = "goods__title"]/text()')).replace(",","")
        sub_x_comm = "".join(sub_x_tree.xpath('//div[@class = "catalog__item__text"]/text()'))
        sub_x_oldp = "".join(sub_x_tree.xpath('//span[@class = "price_block__old-price"]/text()')).replace('.',',')
        sub_x_newp = "".join(sub_x_tree.xpath('//span[@class = "price_block__cur-price"]/text()')).replace('.',',')
        li.append([sub_x_name + sub_x_comm, sub_x_oldp, sub_x_newp])
    
    df = DataFrame(li)
    return df
        
#print get_one_page("http://www.msk.lenta.com/special-nye-predlozheniya/")

In [207]:
def main(url):
    i = 0
    terrs = get_promo_territories(url)
    df = DataFrame()
    for index_terr, row_terr in terrs.iterrows():
        try:
            city = row_terr.ix[2] 
            city_href = row_terr.ix[1]
            #print city, city_href
            hrefs = get_all_pages(city_href)
            for index_cat, row_cat in hrefs.iterrows():
                try:
                    full_href = city_href + row_cat.ix[1]
                    print full_href
                    df_temp = get_one_page(full_href)
                    df_temp['Category'] = "".join(row_cat.ix[0])
                    df_temp['City'] = "".join(city) #join(row_terr.ix[0])
                    df = df.append(df_temp)
                    #print i
                    #i += 1
                except:
                    pass
        except: 
            pass
    return df


In [208]:
df_all = main('http://www.lenta.com')


http://www.lenta.com/kosmetika/
http://www.lenta.com/bytovaya-himiya-tovary-dlya-zhivotnyh/
http://www.lenta.com/tovary-dlya-detey/
http://www.lenta.com/detskaya-odezhda-i-obuv/
http://www.lenta.com/bel-e-i-obuv-dlya-doma/
http://www.lenta.com/zhenskaya-odezhda-7th-june/
http://www.lenta.com/muzhskaya-odezhda-7th-june/
http://www.lenta.com/tovary-dlya-doma-tekstil/
http://www.lenta.com/tovary-dlya-doma-posuda/
http://www.lenta.com/sportivnye-tovary/
http://www.lenta.com/sezonnye-tovary-kancelyariya/
http://www.lenta.com/bytovaya-tehnika/
http://www.lenta.com/mul-timedia/
http://www.lenta.com/avtotovary-instrumenty/
http://www.lenta.com/ovoschi-frukty/
http://www.lenta.com/myaso-vypechka-kulinariya/
http://www.lenta.com/gastronom/
http://www.lenta.com/moreprodukty/
http://www.lenta.com/molochnye-produkty-zamorozhennye-produkty/
http://www.lenta.com/konditerskie-izdeliya/
http://www.lenta.com/chay-kofe-torty-hleb/
http://www.lenta.com/bakaleya/
http://www.lenta.com/napitki/
http://www.le

In [209]:
#print str(time.strftime("%Y-%m-%d"))

#print df_all
adress = str(time.strftime("%Y-%m-%d")) + '_py.xlsx'
print adress
writer = pandas.ExcelWriter(adress) #, engine='xlsxwriter')
df_all.to_excel(writer, sheet_name = "Final")
writer.save()

#df_all.to_excel('')